-todo-
clean the df some
fix dtypes
rename vars to more thought out scheme


In [1]:
import pandas as pd
import json
import numpy as np
import requests
import scrape_steam_games as sgl

In [2]:
import scr_utils as scru
rate_sek=scru.get_sek()

In [3]:
#get the full game list
steam_df=sgl.get_steam_games_list()

-------- filter for specific game ---------

In [4]:
gameName='Hitman 3'
gameNameAlt='Hitman 3'

In [5]:
#exact match
steamGamedf=steam_df.loc[steam_df['name'].str.lower().eq(gameName.lower())]
if steamGamedf.empty:
    steamGamedf=steam_df.loc[steam_df['name'].str.lower().eq(gameNameAlt.lower())]
if steamGamedf.empty:
    #contains match
    steamGamedf=steam_df.loc[steam_df['name'].str.contains(gameName, case=False)]
if steamGamedf.empty:
    #contains match
    steamGamedf=steam_df.loc[steam_df['name'].str.contains(gameNameAlt, case=False)]
steamGamedf

,appid,name
153490,1659040,HITMAN 3


---- get single game info ------

In [6]:
appid_single_game=steamGamedf['appid'].values[0]

url_steam_game = 'https://store.steampowered.com/api/appdetails?appids='+str(appid_single_game)
url_steam_game

'https://store.steampowered.com/api/appdetails?appids=1659040'

In [7]:
#steamGamedf['url'].values[0]

In [8]:
steamResponse = requests.get(url_steam_game).json() 
if steamResponse[str(appid_single_game)]['success']:
    datac=steamResponse[str(appid_single_game)]['data']

In [9]:
#datac

frames=[]
for index, row in steamGamedf.iterrows():
    #print(row["name"], row["url"])
    steamResponse = requests.get(row['url']).json() 
    if steamResponse[str(row["appid"])]['success']:
        datac=steamResponse[str(row["appid"])]['data']
        #if "price_overview" in datac:
        steamSdf=pd.json_normalize(datac)
        steamSdf.keys()
        frames.append(steamSdf)
        
single = pd.concat(frames, ignore_index=True)
single

In [10]:
df_single_game=pd.json_normalize(datac)
df_single_game

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,metacritic.url,recommendations.total,achievements.total,achievements.highlighted,release_date.coming_soon,release_date.date,support_info.url,support_info.email,content_descriptors.ids,content_descriptors.notes
0,game,HITMAN 3,1659040,18,False,full,"[1829591, 1829592, 1829590, 1843460, 1829595, ...","<h1>Standard Edition</h1><p><img src=""https://...",<u><strong>WORLD OF ASSASSINATION</strong></u>...,"Death Awaits. Agent 47 returns in HITMAN 3, th...",...,https://www.metacritic.com/game/pc/hitman-3?ft...,5287,83,"[{'name': 'The Result of Previous Training', '...",False,"20 Jan, 2022",https://www.ioi.dk/player-support/,,"[2, 5]",This Game may contain content not appropriate ...


add read timestamp and add only if later?

In [11]:
df_single_game['date_time'] = pd.Timestamp.now()

In [12]:
if 'price_overview.final' in df_single_game.columns:
    df_single_game = df_single_game.assign(priceSEK=df_single_game["price_overview.final"]/100*rate_sek)
    #df_single_game.round({'priceSEK': 1})
    df_single_game.priceSEK=df_single_game.priceSEK.round()
else:
    print('No price')
df_single_game

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,achievements.total,achievements.highlighted,release_date.coming_soon,release_date.date,support_info.url,support_info.email,content_descriptors.ids,content_descriptors.notes,date_time,priceSEK
0,game,HITMAN 3,1659040,18,False,full,"[1829591, 1829592, 1829590, 1843460, 1829595, ...","<h1>Standard Edition</h1><p><img src=""https://...",<u><strong>WORLD OF ASSASSINATION</strong></u>...,"Death Awaits. Agent 47 returns in HITMAN 3, th...",...,83,"[{'name': 'The Result of Previous Training', '...",False,"20 Jan, 2022",https://www.ioi.dk/player-support/,,"[2, 5]",This Game may contain content not appropriate ...,2022-10-20 12:41:55.029158,263.0


In [13]:
df_single_game.filter(['name','type','controller_support',
       'price_overview.final',
       'price_overview.discount_percent',
       'price_overview.final_formatted','priceSEK', 'date_time'],axis=1)

,name,type,controller_support,price_overview.final,price_overview.discount_percent,price_overview.final_formatted,priceSEK,date_time
0,HITMAN 3,game,full,2399,60,"23,99€",263.0,2022-10-20 12:41:55.029158


In [14]:
df_single_game['priceSEK'].values[0]

263.0

save only if newer and/or not same game

In [15]:
scru.to_paraquet('steam_prices',df_single_game)

pyarrow.Table
type: string
name: string
steam_appid: int64
required_age: string
is_free: bool
controller_support: string
dlc: list<item: int64>
  child 0, item: int64
detailed_description: string
about_the_game: string
short_description: string
supported_languages: string
header_image: string
website: string
legal_notice: string
developers: list<item: string>
  child 0, item: string
publishers: list<item: string>
  child 0, item: string
demos: list<item: struct<appid: int64, description: string>>
  child 0, item: struct<appid: int64, description: string>
      child 0, appid: int64
      child 1, description: string
packages: list<item: int64>
  child 0, item: int64
package_groups: list<item: struct<description: string, display_type: int64, is_recurring_subscription: string, name: string, save_text: string, selection_text: string, subs: list<item: struct<can_get_free_license: string, is_free_license: bool, option_description: string, option_text: string, packageid: int64, percent_savin